### Web Scraping Congress
This file is a script that webscrapes information from voting bills in the US congress during 2020.

We start by importing all needed packages. (Write "!pip install <package>" to install missing packages)

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS

Collect all urls to be scraped!

In [2]:
url_list = []
for i in range(2,254):
    url_list.append(f'https://clerk.house.gov/Votes/2020{i}?Page=2')

Scrape 'em

In [3]:
members = []
parties = []
states = []
votes = []

for i,url in enumerate(url_list):
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')

    member_list = []
    party_list = []
    state_list = []
    vote_list = []

    member_td = soup.find_all('td',attrs={"data-label": "member","hidden":True})
    party_td = soup.find_all('td',attrs={"data-label": "party"})
    state_td = soup.find_all('td',attrs={"data-label": "state"})
    state_td = state_td[1::2]
    vote_td = soup.find_all('td',attrs={"data-label": "vote"})
    for mem_idx in range(len(vote_td)):
        vote_list.append(str(vote_td[mem_idx]).replace('<td data-label="vote">', "").replace('</td>',''))
        state_list.append(str(state_td[mem_idx]).replace('<td class="visible-sm visible-xs" data-label="state">',"").replace('</td>',""))
        party_list.append(str(party_td[mem_idx]).replace('<td data-label="party">','').replace('</td>',''))
        member_list.append(str(member_td[mem_idx]).replace('<td data-label="member" hidden="">','').replace('</td>',''))
    members.append(member_list)
    parties.append(party_list)
    states.append(state_list)
    votes.append(vote_list)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251


In [5]:
all_members = []
for i in members:
    all_members = all_members+i
all_members = np.unique(all_members)

In [6]:
for i, bill in enumerate(members):
    for member in all_members:
        if member not in bill:
            members[i] = sorted(members[i]+[member])
            for pos, dude in enumerate(members[i]):
                if dude == member:
                    votes[i].insert(pos, 'NP')
                    parties[i].insert(pos,'NP')
                    states[i].insert(pos,'NP')

In [7]:
votes = np.array(votes)
members = np.array(members)
states = np.array(states)
parties = np.array(parties)

In [8]:
lm = len(members[0])
graph = np.arange(lm)
graph.shape = (442,1)
missing_data_idx = []
for i in range(len(votes)):
    for j in range(len(votes[i])):
        if votes[i][j] == 'Yea' or votes[i][j] == 'Aye':
            votes[i][j] = 1
        if votes[i][j] == 'Nay' or votes[i][j] == 'No':
            votes[i][j] = 0
        if votes[i][j] == 'Not Voting' or votes[i][j] == 'Present':
            votes[i][j] = 0
            missing_data_idx.append((i,j))
        if votes[i][j] == 'NP':
            votes[i][j] = 0
            missing_data_idx.append((i,j))
    graph = np.concatenate((graph, np.reshape(votes[i], (442, 1))),1)

In [9]:
import numpy as np
import pandas as pd
graph = graph[:,1:]
graph = pd.DataFrame(graph)
graph.to_csv('2020_congress_graph.csv',sep=',')

In [10]:
US_states = np.empty(lm, dtype='<U4')
for i in range(len(states)):
    for j in range(len(states[i])):
        if states[i][j] != 'NP':
            US_states[j] = str(states[i][j])
        

In [11]:
US_parties = np.empty(lm,dtype='<U20')
for i in range(len(parties)):
    for j in range(len(parties[i])):
        if parties[i][j] != 'NP':
            US_parties[j] = str(parties[i][j])

In [34]:
missing_data_idx = np.array(list(map(list,zip(*missing_data_idx))))
missing_data_idx = pd.DataFrame(missing_data_idx)
missing_data_idx.to_csv('2020_congress_missing_data_idx.csv',sep=',')

In [17]:
metadata = np.concatenate((np.reshape(US_states, (442, 1)),np.reshape(US_parties, (442, 1)), np.reshape(members[0], (442,1))),1)

In [18]:
metadata = pd.DataFrame(metadata,columns=['State','Party','Name'])

In [19]:
metadata.to_csv('2020_congress_metadata.csv',sep=',')

In [35]:
graph

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
2,0,0,1,0,0,0,1,0,0,0,...,1,1,1,1,1,0,1,1,1,1
3,1,1,1,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
4,0,0,1,0,0,0,0,1,1,0,...,1,1,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
438,1,1,1,1,1,1,1,0,0,1,...,1,1,1,1,0,1,1,1,1,1
439,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
440,0,0,1,0,0,0,0,0,0,0,...,1,1,1,1,1,0,1,1,0,1
